In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from pathlib import Path
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.data import Dataset

In [2]:
df_sub = pd.read_csv(Path.cwd() / 'data' / 'sample_submission.csv')

In [3]:
df_train = pd.read_csv(Path.cwd() / 'data' / 'train.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_test = pd.read_csv(Path.cwd() / 'data' / 'test.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df_train.drop_duplicates(inplace=True)

In [55]:
dataset = Dataset()

In [56]:
df_train.userid.max()

127495

In [57]:
df_test.userid.max()

127495

In [ ]:
df_train.

In [6]:
train_data, test_data = train_test_split(df_train, random_state=32, shuffle=True)

In [59]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621701 entries, 728049 to 10972
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         621701 non-null  float64
 1   verified        621701 non-null  bool   
 2   reviewTime      621701 non-null  object 
 3   asin            621701 non-null  object 
 4   reviewerName    621580 non-null  object 
 5   reviewText      621485 non-null  object 
 6   summary         621572 non-null  object 
 7   unixReviewTime  621701 non-null  int64  
 8   vote            86973 non-null   object 
 9   style           322200 non-null  object 
 10  image           5277 non-null    object 
 11  userid          621701 non-null  int64  
 12  itemid          621701 non-null  int64  
 13  rating          621701 non-null  float64
dtypes: bool(1), float64(2), int64(3), object(8)
memory usage: 67.0+ MB


In [60]:
train_data.values[0]

array([5.0, True, '08 9, 2016', 'B000UUWECC', 'Dorothy Gomes',
       'Great\nproduct and service.', 'Great product and service',
       1470700800, nan, nan, nan, 3215, 4724, 1.0], dtype=object)

In [61]:
a = set(df_train.userid.values)

In [62]:
len(a)

127448

In [63]:
b = set(df_test.userid.values)

In [27]:
len(b)

109357

In [64]:
df_train [df_train.userid == 5708]

,overall,verified,reviewTime,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,userid,itemid,rating


In [65]:
df_test[df_test.userid == 5708]

,verified,reviewTime,asin,reviewerName,unixReviewTime,vote,style,image,userid,itemid,Id
12710,True,"12 30, 2017",B000JL2LCA,Christopher Norman,1514592000,NaN,NaN,NaN,5708,2936,12710
90429,True,"12 30, 2017",B0001590IC,Christopher Norman,1514592000,NaN,{'Flavor:': ' Rooibos'},NaN,5708,240,90429
157535,True,"06 6, 2018",B00FLA31LE,Christopher Norman,1528243200,NaN,NaN,NaN,5708,25600,157535
192791,True,"09 19, 2016",B00I8VS45G,Christopher Norman,1474243200,NaN,NaN,NaN,5708,27404,192791
267969,True,"12 30, 2017",B01FLKVRC8,Christopher Norman,1514592000,NaN,{'Size:': ' 80 Teabags'},NaN,5708,37854,267969


In [7]:
ratings_coo = sparse.coo_matrix((df_train['rating'].astype(int),
                                 (df_train['userid'],
                                  df_train['itemid'])))

In [8]:
ratings_coo.shape

(127496, 41320)

In [18]:
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 60 #число параметров вектора 
NUM_EPOCHS = 25 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [19]:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)

In [20]:
roc_auc_score(test_data.rating,preds)

0.9998969146294691

In [21]:
preds_sub = model.predict(df_test.userid.values,
                      df_test.itemid.values)

In [11]:
model.predict([128000],[1])

ValueError: The user feature matrix specifies more features than there are estimated feature embeddings: 127496 vs 128001.

In [22]:
len(preds_sub)

285965

In [13]:
preds.min(), preds.max()

NameError: name 'preds' is not defined

In [23]:
normalized_preds = (preds_sub - preds_sub.min())/(preds_sub - preds_sub.min()).max()

In [24]:
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [25]:
df_sub['rating']= normalized_preds

In [26]:
df_sub.to_csv('submission_log.csv', index=False)